In [3]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import joblib
import matplotlib
import operator
# Set 
matplotlib.rcParams['font.serif'] = 'Times New Roman'
matplotlib.rcParams['font.family'] = "serif"
matplotlib.rcParams['font.size'] = 10

# import custom functions
from functions_statistical_performance import *

In [4]:
# True to exactly reproduce the results presented in the paper (same classifiers)
reproduceflag=True
pathdir1='./input_data/'
pathdir2='./paper_classifiers_outcomes/'

# Load Data

In [5]:
RFOFF=pd.read_csv(pathdir1+'OpenFoodFacts_10nutrients.csv')

In [6]:
RFOFF

,_id,product_name,brands,nova_group,scans_n,additives_n,additives_original_tags,categories,len_countries,USA,...,sugars_100g,fiber_100g,calcium_100g,iron_100g,sodium_100g,vitamin-c_100g,cholesterol_100g,saturated-fat_100g,vitamin-a_100g,trans-fat_100g
0,4.530000e+03,Banana Chips Sweetened (Whole),NaN,4,NaN,0.0,[],NaN,1.0,1.0,...,14.29,3.6,0.00000,0.00129,0.000000,0.0214,0.0180,28.57,0.00000,0.000
1,3.364000e+04,Granola Cranberry And Acai,Viva La Salud,4,NaN,0.0,[],NaN,1.0,1.0,...,25.45,5.5,0.03600,0.00262,0.101600,0.0000,0.0000,0.91,0.00000,0.000
2,3.368800e+04,"Peanuts, Mixed Nuts",Northgate Market,4,NaN,1.0,['en:e415'],NaN,1.0,1.0,...,14.29,7.1,0.07100,0.00514,0.217424,0.0000,0.0000,7.14,0.00000,0.000
3,3.626900e+04,Milk Chocolate Peanut Butter Malt Balls,Sunridge,4,NaN,3.0,"['en:e322i', 'en:e500ii', 'en:e471']",NaN,1.0,1.0,...,57.50,2.5,0.10000,0.00090,0.038608,NaN,0.0120,20.00,NaN,NaN
4,4.359500e+04,Cranberries,Torn & Glasser,3,NaN,0.0,[],"Plant-based foods and beverages, Plant-based f...",1.0,1.0,...,66.67,10.0,0.00000,0.00000,0.000000,0.0000,0.0000,0.00,0.00000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233826,6.811310e+11,Triple Chocolate Brownie,Marketside,4,NaN,5.0,"['en:e322i', 'en:e471', 'en:e211', 'en:e330', ...","Desserts,Biscuits and cakes,Cakes,Chocolate ca...",1.0,1.0,...,39.20,2.7,0.05410,0.00135,0.203000,NaN,0.0676,12.20,NaN,0.000
233827,7.874222e+10,Chicken egg roll,Walmart,4,NaN,3.0,"['en:e330', 'en:e412', 'en:e322i']",Egg Roll,1.0,1.0,...,3.75,1.0,0.03438,0.00110,0.562500,NaN,0.0100,1.25,NaN,0.000
233828,6.038308e+10,Crisp & Crunchy Spicy Southern-Style Chicken W...,President's Choice,4,NaN,9.0,"['en:e322i', 'en:e433', 'en:e415', 'en:e412', ...",NaN,1.0,NaN,...,1.83,0.0,0.01830,0.00132,0.450000,NaN,0.0688,4.59,NaN,0.183
233829,7.709150e+12,Yogurt Fresa,Prílac,4,NaN,4.0,"['en:e14xx', 'en:e331', 'en:e129', 'en:e102']","Lácteos,Comidas fermentadas,Productos fermenta...",1.0,NaN,...,16.50,0.0,0.10000,0.00000,0.045000,0.0012,0.0100,1.50,0.00003,NaN


# Missing Data in Nutrition Facts

In [7]:
nl=['proteins_100g',
 'fat_100g',
 'carbohydrates_100g',
 'sugars_100g',
 'fiber_100g',
 'calcium_100g',
 'iron_100g',
 'sodium_100g',
 'vitamin-c_100g',
 'cholesterol_100g',
 'saturated-fat_100g',
 'vitamin-a_100g',
 'trans-fat_100g']
RFOFF[nl].isna().sum()

proteins_100g             0
fat_100g                  0
carbohydrates_100g        0
sugars_100g               0
fiber_100g                0
calcium_100g              0
iron_100g                 0
sodium_100g               0
vitamin-c_100g        44555
cholesterol_100g          0
saturated-fat_100g        0
vitamin-a_100g        46099
trans-fat_100g         5142
dtype: int64

# Keep 11 nutrients including Trans-Fats

In [8]:
RFOFF=RFOFF[RFOFF['trans-fat_100g'].notna()]
# select the right nutrients
nl=['proteins_100g',
 'fat_100g',
 'carbohydrates_100g',
 'sugars_100g',
 'fiber_100g',
 'calcium_100g',
 'iron_100g',
 'sodium_100g',
 'cholesterol_100g',
 'saturated-fat_100g',
 'trans-fat_100g']

RFOFF[nl]=RFOFF[nl].apply(np.log).replace(-np.inf,-20)

/Users/giulia/anaconda/lib/python3.6/site-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [9]:
RFOFF['pythonlabel']=(RFOFF['nova_group']-1)
X=RFOFF.loc[:, nl].copy()
y=RFOFF['pythonlabel'].copy()

/Users/giulia/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [10]:
if reproduceflag:
    (performancesAUC11P, performancesAUP11P, splits11P, model_per_fold11P)=joblib.load(pathdir2+'OpenFoodFacts11p_Performance.pkl')
    print("AUC: average over the folds", performancesAUC11P.mean(axis=0))
    print("AUC: std over the folds", performancesAUC11P.std(axis=0))
    print("AUP: average over the folds", performancesAUP11P.mean(axis=0))
    print("AUP: std over the folds", performancesAUP11P.std(axis=0))
else:
    smoteflag=False
    (performancesAUC11P, performancesAUP11P, splits11P, model_per_fold11P)=AUCAUPkfold(X,y,smoteflag);

/Users/giulia/anaconda/lib/python3.6/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.23.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/giulia/anaconda/lib/python3.6/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.23.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


AUC: average over the folds [0.98798905 0.9860438  0.93203265 0.95076783]
AUC: std over the folds [0.00064719 0.00447191 0.00152541 0.00085953]
AUP: average over the folds [0.94917486 0.85326995 0.82867634 0.9724672 ]
AUP: std over the folds [0.00269426 0.01671317 0.00275089 0.00071206]


# Including Additives Information

In [11]:
X=RFOFF.loc[:, nl+['additives_n']].copy()
X.fillna(0,inplace=True)
y=RFOFF['pythonlabel'].copy()

In [12]:
# keep same splits as before
(performancesAUC11PA, performancesROC11PA, mean_fpr11PA, mean_tpr11PA, std_tpr11PA, tprs_upper11PA, tprs_lower11PA)=ROCkfold(X,y,splits11P)


AUC: average over the folds
[0.99255319 0.98778501 0.96530099 0.97824244]
AUC: std over the folds
[0.00032709 0.00468209 0.00096202 0.00068267]


In [13]:
(performancesAUP11PA, performancesPrecisionRecall11PA, mean_recall11PA, mean_precision11PA, std_precision11PA, precision_upper11PA, precision_lower11PA)=PrecisionRecallCurvekfold(X,y,splits11P)


AUP: average over the folds
[0.9629157  0.89025262 0.89563901 0.98872219]
AUP: std over the folds
[0.00158259 0.01305424 0.00298316 0.00038775]


# Using just the Number of Additives

In [14]:
# keep same splits as before
(performancesAUCna, performancesROCna, mean_fprna, mean_tprna, std_tprna, tprs_upperna, tprs_lowerna)=ROCkfold(X[['additives_n']],y,splits11P)

AUC: average over the folds
[0.85992325 0.83202339 0.82025567 0.90730657]
AUC: std over the folds
[0.00048496 0.00337543 0.00069272 0.00060892]


In [15]:
(performancesAUPna, performancesPrecisionRecallna, mean_recallna, mean_precisionna, std_precisionna, precision_upperna, precision_lowerna)=PrecisionRecallCurvekfold(X[['additives_n']],y,splits11P)

AUP: average over the folds
[0.28879856 0.02622742 0.43920244 0.94154806]
AUP: std over the folds
[0.00092989 0.00049781 0.00126893 0.00035017]
